<a href="https://colab.research.google.com/github/StickMonkey615/JHCSMod4/blob/main/Copy_of_KSVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 4 Guidance

This notebook is a template for module 4b and 4c, which will be tested in Google Colab, your code needs to run there.
The structure has been provided to improve consistency and make it easier for markers to understand your code but still give students the flexibility to be creative.  You need to populate the required functions to solve this problem.  All dependencies should be documented in the next cell.

You can:
    add further cells or text blocks to extend or further explain your solution
    add further functions

Dont:
    rename functions
   

In [3]:
# Fixed dependencies - do not remove or change.
import pytest
import pandas as pd
import numpy as np
'''from google.colab import drive'''
import tensorflow as tf
# drive.mount('/content/gdrive/')
# Import your dependencies
#!pip install xlrd>=1.2.0
#!pip install --upgrade xlrd
#!apt-get -qq install -y graphviz && pip install xlrd
import xlrd



In [4]:
# Import data

def import_local_data(file_path):
    """This function needs to import the data file into collab and return a pandas dataframe
    """
    raw_df = pd.read_excel(file_path)
    return raw_df 

In [5]:
#local_file_path = "C:/Users/shaun/Python_work/jHub Coding Scheme/Module 4 - ML/breast-cancer.xls"
local_file_path = "breast-cancer.xls"

In [6]:
# Dont change
raw_data = import_local_data(local_file_path)

### Conduct exploratory data analysis and explain your key findings - Examine the data, explain its key features and what they look like.  Highlight any fields that are anomalous.

In [ ]:
# Explain your key findings

In [7]:
def process_data(unproc_data):
    
    # Taking care of missing data
    # from sklearn.impute import SimpleImputer
    # imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    # imputer.fit(raw_data[:, 5])
    # X[:, 5] = imputer.transform(X[:, 5])
    # print(X)

    # Remove non-categorical data
    dm = unproc_data.pop('deg-malig')

    # Correct date types in tumor-size abd inv-nodes variables
    for i in range(0, len(unproc_data)):
        if type(unproc_data['tumor-size'][i]) is not str:
            if unproc_data['tumor-size'][i].day == 1:
                unproc_data['tumor-size'][i] = str(unproc_data['tumor-size'][i].month) + '-' + str(unproc_data['tumor-size'][i].year-2000)
            else:
                unproc_data['tumor-size'][i] = str(unproc_data['tumor-size'][i].day) + '-' + str(unproc_data['tumor-size'][i].month)
        if type(unproc_data['inv-nodes'][i]) is not str:
            if unproc_data['inv-nodes'][i].day == 1:
                unproc_data['inv-nodes'][i] = str(unproc_data['inv-nodes'][i].month) + '-' + str(unproc_data['inv-nodes'][i].year-2000)
            else:
                unproc_data['inv-nodes'][i] = str(unproc_data['inv-nodes'][i].day) + '-' + str(unproc_data['inv-nodes'][i].month)

    # Convert node-caps and irradiat variables into booleans
    for i in range(0, len(unproc_data)):
        if unproc_data['node-caps'][i] == 'yes':
            unproc_data['node-caps'][i] = True
        elif unproc_data['node-caps'][i] == 'no':
            unproc_data['node-caps'][i] = False
        else:
            unproc_data['node-caps'][i] = False
        if unproc_data['irradiat'][i] == 'yes':
            unproc_data['irradiat'][i] = True
        elif unproc_data['irradiat'][i] == 'no':
            unproc_data['irradiat'][i] = False
        else:
            unproc_data['irradiat'][i] = False


    # Encode the catagorical data (dummy variables)
    proc_data = pd.get_dummies(data=unproc_data, prefix_sep='_', drop_first=True)
    
    # Add back in non-categorical data
    proc_data.insert(0, 'deg-malig', dm)
    
    return proc_data

In [8]:
clean_data = process_data(raw_data)

Create any data pre-processing that you will conduct on seen and unseen data.  Regardless of the model you use, this dataframe must contain only numeric features and have a strategy for any expected missing values. Any objects can that are needed to handle the test data that are dependent on the training data can be stored in the model class.  You are recommended to use sklearn Pipelines or similar functionality to ensure reproducibility.

In [9]:
# Split your data so that you can test the effectiveness of your model
# Split the data into a Training set and a Test set
dfs = np.split(clean_data, [len(clean_data.columns)-1], axis=1)
X = dfs[0]
y = dfs[1]
#X = raw_data.iloc[:, :-1].values
#y = raw_data.iloc[:, -1].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
# Populate preprocess_training_data and preprocess_test_data to preprocess data.
# You must process test and train separately so your model does not accidently gain information that a model wouldnt have in reality and therefore get better predictions
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
class Module4_Model:
    
    def __init__(self):
        self.model = tf.keras.models.Sequential()
        self.model.add(tf.keras.layers.Dense(units=6, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=6, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
        
    def preprocess_training_data(self, training_df):
        """
        This function should process the training data and store any features required in the class
        """
        processed_df = training_df
        return processed_df

    def preprocess_test_data(self, test_df):
        processed_df = test_df
        return processed_df


In [12]:
# Dont change
my_model = Module4_Model()

In [14]:
# Dont change
x_train_processed = my_model.preprocess_training_data(X_train)

In [ ]:
# Create a model

In [20]:
my_model.model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [21]:
# Dont change
x_test_processed = my_model.preprocess_test_data(X_test)

In [22]:
# Train your model
my_model.model.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
8/8 [==============================] - 2s 9ms/step - loss: 0.6691 - accuracy: 0.6140
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 0.6554 - accuracy: 0.6272
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6440 - accuracy: 0.6535
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6343 - accuracy: 0.6579
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: 0.6263 - accuracy: 0.6579
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 0.6181 - accuracy: 0.6667
Epoch 7/100
8/8 [==============================] - 0s 4ms/step - loss: 0.6114 - accuracy: 0.6711
Epoch 8/100
8/8 [==============================] - 0s 4ms/step - loss: 0.6042 - accuracy: 0.6798
Epoch 9/100
8/8 [==============================] - 0s 4ms/step - loss: 0.5981 - accuracy: 0.6842
Epoch 10/100
8/8 [==============================] - 0s 3ms/step - loss: 0.5920 - accuracy: 0.6886
Epoch 11/100
8/8 [===========

8/8 [==============================] - 0s 4ms/step - loss: 0.4016 - accuracy: 0.8158
Epoch 85/100
8/8 [==============================] - 0s 3ms/step - loss: 0.4002 - accuracy: 0.8158
Epoch 86/100
8/8 [==============================] - 0s 3ms/step - loss: 0.3984 - accuracy: 0.8158
Epoch 87/100
8/8 [==============================] - 0s 3ms/step - loss: 0.3967 - accuracy: 0.8246
Epoch 88/100
8/8 [==============================] - 0s 3ms/step - loss: 0.3952 - accuracy: 0.8289
Epoch 89/100
8/8 [==============================] - 0s 3ms/step - loss: 0.3936 - accuracy: 0.8289
Epoch 90/100
8/8 [==============================] - 0s 3ms/step - loss: 0.3918 - accuracy: 0.8289
Epoch 91/100
8/8 [==============================] - 0s 4ms/step - loss: 0.3901 - accuracy: 0.8333
Epoch 92/100
8/8 [==============================] - 0s 3ms/step - loss: 0.3887 - accuracy: 0.8377
Epoch 93/100
8/8 [==============================] - 0s 3ms/step - loss: 0.3872 - accuracy: 0.8377
Epoch 94/100
8/8 [===============

In [ ]:
# use your model to make a prediction on unseen data

In [23]:
y_pred = my_model.model.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

2/2 [==============================] - 0s 19ms/step


AttributeError: 'DataFrame' object has no attribute 'reshape'

In [27]:
# Asssess the accuracy of your model and explain your key findings
# Generate confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[33  3]
 [20  2]]


0.603448275862069

### Unit tests:

#### Checking training and test data for null values. This will work for both pd dataframes and np arrays, and ensures no null values exist.

In [28]:
def test_no_nulls(data):
    """ Assert no null values within pd dataframe or np array """
    
    # if data is numpy array, handle accordingly
    if isinstance(data, (np.ndarray)):
        assert not np.isnan(np.min(data))
    
    # if not np array, assume data is pandas dataframe
    else:
        assert data.isna().sum().sum() == 0

In [29]:
# run null data unit test on both training and test data
test_no_nulls(x_train_processed)
test_no_nulls(x_test_processed)